# Estimation of failure probability due to wave run-up (Hunt's equation)

In this example, we will demonstrate the application of various reliability methods (`directional_sampling`, `importance_sampling`, `adaptive_importance_sampling`, `subset_simulation`) to estimate the probability of failure of a levee due to wave run-up. The failure mechanism is characterized by the Hunt's equation, which is a simple model to estimate the wave run-up.

### Define model

First, we import the necessary packages:

In [1]:
from probabilistic_library import ReliabilityProject, DistributionType, ReliabilityMethod, SampleMethod

We consider the following limit state function:

$Z = h_{crest} - (h + R_u)$

where: <br>

$h_{crest}$ is the crest height of the flood defence (m) <br>
$h$ is the water level on the flood defence (m) <br>
$R_u$ represents the wave run-up (m)

The wave run-up $R_u$ is derived using the Hunt's equation:

$R_u = \xi \cdot H_s$

with:

$\xi = \frac{\tan (\alpha)}{\sqrt{2\cdot \pi \cdot H_s/L_0}} $

$L_0 = g \cdot T_p^2$

where:

$H_s$ is the significant wave height (m) <br>
$T_p$ is the peak wave period (s) <br>
$g$ is the gravitational acceleration ($9.81 m/s ^2$) <br>
$\tan (\alpha)$ is the slope of the flood defence (-)

The parameters $h$, $H_s$ and $T_p$ represent the imposed load, while $h_{crest}$ and $\tan (\alpha)$ stand for the strength of the levee.

In [2]:
from utils.models import hunt

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [3]:
project = ReliabilityProject()
project.model = hunt
project.model.print()

Model hunt:
Input parameters:
  t_p
  tan_alpha
  h_s
  h_crest
  h
Output parameters:
  Z


We assume the following distributions for the parameters present in the limit state function:

In [4]:
project.variables["t_p"].distribution = DistributionType.log_normal
project.variables["t_p"].mean = 6
project.variables["t_p"].deviation = 2

project.variables["tan_alpha"].distribution = DistributionType.deterministic
project.variables["tan_alpha"].mean = 0.333333

project.variables["h_s"].distribution = DistributionType.log_normal
project.variables["h_s"].mean = 3
project.variables["h_s"].deviation = 1

project.variables["h_crest"].distribution = DistributionType.log_normal
project.variables["h_crest"].mean = 10
project.variables["h_crest"].deviation = 0.05

project.variables["h"].distribution = DistributionType.exponential
project.variables["h"].shift = 0.5
project.variables["h"].scale = 1

### Perform reliability calculations with Directional Sampling

We start with the reliability method `directional_sampling`. The reliability analysis is executed using `project.run()`, and the results are accessed from `project.design_point`.

In [5]:
project.settings.reliability_method = ReliabilityMethod.directional_sampling
project.settings.minimum_directions = 10000
project.settings.maximum_directions = 20000
project.settings.variation_coefficient = 0.02

project.run()
project.design_point.print()

Reliability (Directional Sampling)
 Reliability index = 1.8711604829760653
 Probability of failure = 0.030661420137601952
 Convergence = 0.019998996781194246 (converged)
 Model runs = 82734
Alpha values:
 t_p: alpha = -0.759946009166965, x = 9.030827950876963
 tan_alpha: alpha = 0.0, x = 0.333333
 h_s: alpha = -0.38819385340233, x = 3.6027862684699463
 h_crest: alpha = 0.013262417979168769, x = 9.998634296961573
 h: alpha = -0.5211637972856513, x = 2.303415838909463



### Perform reliability calculations with Importance Sampling

We now conduct the reliability analysis using the `importance_sampling`.

In [6]:
project.settings.reliability_method = ReliabilityMethod.importance_sampling
project.settings.minimum_samples = 1000
project.settings.maximum_samples = 100000
project.settings.variation_coefficient = 0.02

project.settings.stochast_settings["t_p"].start_value = 1
project.settings.stochast_settings["h_s"].start_value = 1
project.settings.stochast_settings["h"].start_value = 1

project.run()
project.design_point.print()

#reset values
project.settings.stochast_settings["t_p"].start_value = 0
project.settings.stochast_settings["h_s"].start_value = 0
project.settings.stochast_settings["h"].start_value = 0

Reliability:
 Reliability index = 1.8828282721688645
 Probability of failure = 0.02986182214216561
 Convergence = 0.01999995907886772 (converged)
 Model runs = 39776
Alpha values:
 t_p: alpha = -0.763485147061822, x = 9.076468038765848
 tan_alpha: alpha = 0.0, x = 0.333333
 h_s: alpha = -0.38666194747412574, x = 3.6047105651917946
 h_crest: alpha = 0.0206070503142823, x = 9.997935250029252
 h: alpha = -0.5168735997019476, x = 2.300411544327761



### Perform reliability calculations with Adaptive Importance Sampling

We now conduct the reliability analysis using the `adaptive_importance_sampling` method.

In [7]:
project.settings.reliability_method = ReliabilityMethod.adaptive_importance_sampling
project.settings.minimum_samples = 10000
project.settings.maximum_samples = 100000
project.settings.minimum_variance_loops = 5
project.settings.maximum_variance_loops = 10
project.settings.fraction_failed = 0.5
project.settings.variation_coefficient = 0.02

project.run()
project.design_point.print()

Reliability:
 Reliability index = 1.8800015664915963
 Probability of failure = 0.030053932215033337
 Convergence = 0.019998432841262095 (converged)
 Model runs = 65024
Alpha values:
 t_p: alpha = -0.7655055789023633, x = 9.081301809546584
 tan_alpha: alpha = 0.0, x = 0.333333
 h_s: alpha = -0.38014700983128086, x = 3.5891344163464347
 h_crest: alpha = 0.008789833715386875, x = 9.999048806910018
 h: alpha = -0.5190493217498215, x = 2.3043679996051454

Contributing design points:
  Reliability (Variance loop 1)
   Reliability index = 1.882973723851801
   Probability of failure = 0.029851964496731522
   Convergence = 0.06592288616825591 (not converged)
   Model runs = 10001
  Alpha values:
   t_p: alpha = -0.7576118270020167, x = 9.04427000418764
   tan_alpha: alpha = 0.0, x = 0.333333
   h_s: alpha = -0.3744818273322283, x = 3.578040301716594
   h_crest: alpha = 0.022719276944657803, x = 9.997736281115236
   h: alpha = -0.5341081491979816, x = 2.3497456912166212

  Reliability (Variance 

### Perform reliability calculations with Subset Simulation

We now conduct the reliability analysis using the `subset_simulation` method.

In [8]:
project.settings.reliability_method = ReliabilityMethod.subset_simulation
project.settings.minimum_samples = 1000
project.settings.maximum_samples = 50000
project.settings.variation_coefficient = 0.02
project.settings.sample_method = SampleMethod.adaptive_conditional

project.run()
project.design_point.print()

Reliability:
 Reliability index = 1.8810876257425375
 Probability of failure = 0.029980000000000024
 Convergence = 0.006834554905808205 (not converged)
 Model runs = 45000
Alpha values:
 t_p: alpha = -0.7686904565976019, x = 9.101434711035953
 tan_alpha: alpha = 0.0, x = 0.333333
 h_s: alpha = -0.38319361764655147, x = 3.5962991283918497
 h_crest: alpha = 0.019079768958409547, x = 9.998080661130349
 h: alpha = -0.5117749463848467, x = 2.2846805335478786

Contributing design points:
  Reliability (Subset iteration 1)
   Reliability index = 1.8831503161144236
   Probability of failure = 0.02984000000000003
   Convergence = 0.025499829153609457 (not converged)
   Model runs = 50001
  Alpha values:
   t_p: alpha = -0.77134935981625, x = 9.120932074214549
   tan_alpha: alpha = 0.0, x = 0.333333
   h_s: alpha = -0.36973379618867896, x = 3.5677475077585328
   h_crest: alpha = 0.04009905175136397, x = 9.996100158696631
   h: alpha = -0.5164389132468732, x = 2.2994311147119264

